In [19]:
import tensorflow as tf
from tensorflow import keras
import numpy as np
np.random.seed(521)
tf.random.set_seed(1314)
pi = tf.constant(np.pi, dtype=tf.float64)

class PosPINN(keras.Sequential):
    def __init__(self, Layers, name=None):
        super(PosPINN, self).__init__(name=name)
        self.add(keras.Input(shape=(Layers[0],), dtype=tf.float64))
        for i in range(1, len(Layers)-1):
            self.add(keras.layers.Dense(Layers[i], dtype=tf.float64, activation='tanh'))
        self.add(keras.layers.Dense(Layers[-1], dtype=tf.float64, name="outputs"))
        
        # 添加用于权重的变量
        self.u_weights = self.add_weight(shape=(1,), initializer='ones', dtype=tf.float64, trainable=True)
        self.f_weights = self.add_weight(shape=(1,), initializer='ones', dtype=tf.float64, trainable=True)
    
    @tf.function
    def loss_U(self, X_u_train, u_train):
        u = self(X_u_train)
        loss_u = tf.reduce_mean(tf.square(u_train - u))
        return loss_u
    
    @tf.function
    def loss_PDE(self, X_f_train,X_b_train):
        x = X_f_train[:, 0:1]
        y = X_f_train[:, 1:2]
        with tf.GradientTape(persistent=True) as tape:
            tape.watch([x, y])
            X = tf.concat([x, y], axis=1)
            u = self(X)
            u_x = tape.gradient(u, x)
            u_y = tape.gradient(u, y)
            u_xx = tape.gradient(u_x, x)
            u_yy = tape.gradient(u_y, y)
        u_i = self(X_b_train)
        loss_f1= u_xx + u_yy +2*pi**2*tf.sin(pi*x)*tf.sin(pi*y)
        loss_f1 = tf.reduce_mean(tf.square(loss_f1))
        loss_f2 =tf.reduce_mean(tf.square(u_i))
        loss_f = loss_f1 + loss_f2
        return loss_f
    
    @tf.function
    def train_step(self, X_u_train, u_train, X_i_train):
        loss=self.loss_U(X_u_train, u_train) + self.loss_PDE(X_u_train,X_i_train)
        with tf.GradientTape() as tape:
            a=tf.math.sigmoid(2*self.u_weights+0.5)
            b=tf.math.sigmoid(2*self.f_weights+0.5)
            loss_r = a * self.loss_U(X_u_train, u_train) + b * self.loss_PDE(X_u_train, X_i_train)
        gradients = tape.gradient(loss_r, self.trainable_variables)
        self.optimizer.apply_gradients(zip(gradients, self.trainable_variables))
        return loss
    
    @tf.function
    def train(self, X_u_train, u_train, X_i_train, epochs=200):
        n_u = X_u_train.shape[0]
        n_f = X_i_train.shape[0]
        for epoch in tf.range(1, epochs+1):
            loss = self.train_step(X_u_train, u_train, X_i_train)
            if epoch % 50 == 0:
                tf.print("Training loss (every 50 epochs) at epoch", epoch, ":", loss)
                loss_per50.append(loss)
            if epoch % 1 == 0:
                with tf.GradientTape(persistent=True) as tape:
                    tape.watch([self.u_weights, self.f_weights])
                    a=tf.math.sigmoid(2*self.u_weights+0.5)
                    b=tf.math.sigmoid(2*self.f_weights+0.5)
                    loss_U = a* self.loss_U(X_u_train, u_train)
                    loss_PDE = b * self.loss_PDE(X_u_train, X_i_train)
                    total_loss = loss_U + loss_PDE

                    du_dw = -tape.gradient(total_loss, self.u_weights)
                    df_dw = -tape.gradient(total_loss, self.f_weights)
                    self.optimizer.apply_gradients(zip([du_dw, df_dw], [self.u_weights, self.f_weights])) 
# 定义迪利克雷边界条件热传导方程的参数和边界条件函数
u_exact = lambda x, y: np.sin(np.pi * x) * np.sin(np.pi * y)  # 精确解
loss_per50=[]
# 生成训练数据
n_u = 5000  # 内部节点数量
n_i = 5000  # 边界节点数量
X_u_train = np.random.uniform(low=0, high=1, size=(n_u, 2))
u_train = u_exact(X_u_train[:, 0:1], X_u_train[:, 1:2])
X_i_train_1 = np.random.uniform(low=0, high=1, size=(1250, 2))
X_i_train_1[:, 0] = 1
X_i_train_2 = np.random.uniform(low=0, high=1, size=(1250, 2))
X_i_train_2[:, 0] = 0
X_i_train_3 = np.random.uniform(low=0, high=1, size=(1250, 2))
X_i_train_3[:, 1] = 1
X_i_train_4 = np.random.uniform(low=0, high=1, size=(1250, 2))
X_i_train_4[:, 1] = 0
X_i_train = np.concatenate((X_i_train_1, X_i_train_2, X_i_train_3, X_i_train_4), axis=0)
X_i_train = tf.convert_to_tensor(X_i_train, dtype=tf.float64)
# 创建 PINN 模型并进行训练
Layers = [2, 40, 40,40, 1]  # 神经网络层结构
model = PosPINN(Layers)
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.005))
model.train(X_u_train, u_train, X_i_train, epochs=1000)
n_pred = 5000  # 预测集中的点数量

x_pred = np.random.uniform(low=0, high=1, size=(n_pred, 2))

# 将预测集转换为 tf.Tensor 类型
x_pred_tf = tf.convert_to_tensor(x_pred, dtype=tf.float64)

# 使用模型预测
y_pred = model.predict(x_pred_tf)

# 将预测结果转换为 numpy 数组类型
file_path ="D:\Desktop\graduate paper\spaw pos pinn data_pred.npz"
np.savez(file_path, x_pred=x_pred, y_pred=y_pred)


Training loss (every 50 epochs) at epoch 50 : 4.4729138693483277
Training loss (every 50 epochs) at epoch 100 : 0.57409452738724
Training loss (every 50 epochs) at epoch 150 : 0.17857785757244726
Training loss (every 50 epochs) at epoch 200 : 0.098993621337223225
Training loss (every 50 epochs) at epoch 250 : 0.046512208660267909
Training loss (every 50 epochs) at epoch 300 : 0.039872753606135257
Training loss (every 50 epochs) at epoch 350 : 0.033117661156683811
Training loss (every 50 epochs) at epoch 400 : 0.029243543690998242
Training loss (every 50 epochs) at epoch 450 : 0.034545980730948805
Training loss (every 50 epochs) at epoch 500 : 0.023809218811457204
Training loss (every 50 epochs) at epoch 550 : 0.021536771182965438
Training loss (every 50 epochs) at epoch 600 : 0.031755444263326743
Training loss (every 50 epochs) at epoch 650 : 0.018307818308101138
Training loss (every 50 epochs) at epoch 700 : 0.016711043781365082
Training loss (every 50 epochs) at epoch 750 : 0.2658789